# Wavelet Transform Harmonics Detector for Underwater Acoustic Signals

This notebook implements advanced harmonics detection using continuous wavelet transforms and cyclo-stationary analysis for underwater acoustic target recognition, specifically designed for scooter/DPV detection.

## Implementation Overview
- **Continuous Wavelet Transform (CWT)** for time-frequency analysis
- **Cyclo-stationary Analysis** for periodic pattern detection  
- **Multi-scale Harmonics Detection** with validation
- **Advanced Visualization** of detected patterns

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
import librosa
import soundfile as sf
import pywt
from scipy.signal import find_peaks, butter, filtfilt
from scipy.ndimage import gaussian_filter
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

In [2]:
class WaveletHarmonicsDetector:
    """
    Advanced harmonics detector using continuous wavelet transforms and cyclo-stationary analysis
    for underwater acoustic target recognition.
    """
    
    def __init__(self, sample_rate=192000):
        self.sample_rate = sample_rate
        self.audio_data = None
        self.cwt_coeffs = None
        self.frequencies = None
        self.time_vector = None
        self.detected_harmonics = []
        
    def load_and_preprocess(self, filepath, duration=None, bandpass_filter=True, 
                           low_freq=10, high_freq=20000):
        """
        Load audio file and apply preprocessing
        
        Parameters:
        filepath: path to audio file
        duration: maximum duration to load (seconds)
        bandpass_filter: apply bandpass filtering
        low_freq, high_freq: bandpass filter frequencies
        """
        try:
            # Load audio file
            self.audio_data, sr = librosa.load(filepath, sr=self.sample_rate, duration=duration)
            print(f"Loaded audio: {len(self.audio_data)/sr:.2f}s, {sr}Hz, {len(self.audio_data)} samples")
            
            # Apply bandpass filter to remove DC and high-frequency noise
            if bandpass_filter:
                nyquist = sr / 2
                low = low_freq / nyquist
                high = min(high_freq / nyquist, 0.99)
                
                b, a = butter(4, [low, high], btype='band')
                self.audio_data = filtfilt(b, a, self.audio_data)
                print(f"Applied bandpass filter: {low_freq}-{high_freq} Hz")
                
            # Create time vector
            self.time_vector = np.arange(len(self.audio_data)) / sr
            
            return True
            
        except Exception as e:
            print(f"Error loading audio file: {e}")
            return False

In [ ]:
def compute_cwt(self, wavelet='cmor1.5-1.0', scales=None, freq_range=(20, 5000)):
    """
    Compute Continuous Wavelet Transform
    
    Parameters:
    wavelet: mother wavelet ('cmor' for complex Morlet, 'mexh' for Mexican hat)
    scales: array of scales for CWT
    freq_range: frequency range of interest (Hz)
    """
    if self.audio_data is None:
        print("No audio data loaded. Call load_and_preprocess() first.")
        return False
        
    # Generate scales corresponding to desired frequency range
    if scales is None:
        min_freq, max_freq = freq_range
        # Convert frequencies to scales (approximate for complex Morlet)
        max_scale = self.sample_rate / (2 * min_freq)
        min_scale = self.sample_rate / (2 * max_freq)
        scales = np.logspace(np.log10(min_scale), np.log10(max_scale), 100)
        
    print(f"Computing CWT with {len(scales)} scales...")
    
    # Compute CWT
    self.cwt_coeffs, self.frequencies = pywt.cwt(
        self.audio_data, scales, wavelet, sampling_period=1/self.sample_rate
    )
    
    print(f"CWT computed: {self.cwt_coeffs.shape[0]} frequencies x {self.cwt_coeffs.shape[1]} time points")
    print(f"Frequency range: {self.frequencies.min():.1f} - {self.frequencies.max():.1f} Hz")
    
    return True

# Add method to the class
WaveletHarmonicsDetector.compute_cwt = compute_cwt

IndentationError: unexpected indent (3567302490.py, line 1)

In [ ]:
    def detect_harmonics(self, min_amplitude=0.1, min_duration=0.5, harmonic_tolerance=0.05):
        """
        Detect harmonic series in the CWT coefficients
        
        Parameters:
        min_amplitude: minimum amplitude relative to maximum (0-1)
        min_duration: minimum duration for persistent harmonics (seconds)
        harmonic_tolerance: tolerance for harmonic frequency ratios (0-1)
        """
        if self.cwt_coeffs is None:
            print("No CWT coefficients. Call compute_cwt() first.")
            return False
            
        # Get magnitude of CWT coefficients
        magnitude = np.abs(self.cwt_coeffs)
        
        # Apply smoothing to reduce noise
        magnitude_smooth = gaussian_filter(magnitude, sigma=1.0)
        
        # Find peaks in each time slice
        harmonics_data = []
        
        dt = self.time_vector[1] - self.time_vector[0]
        min_time_points = int(min_duration / dt)
        
        print(f"Detecting harmonics with min amplitude: {min_amplitude:.2f}, min duration: {min_duration}s")
        
        # Process in chunks to find persistent peaks
        chunk_size = max(min_time_points, 100)
        
        for i in range(0, magnitude_smooth.shape[1] - chunk_size, chunk_size // 2):
            # Get time chunk
            chunk_mag = magnitude_smooth[:, i:i+chunk_size]
            time_chunk = self.time_vector[i:i+chunk_size]
            
            # Find average spectrum for this chunk
            avg_spectrum = np.mean(chunk_mag, axis=1)
            
            # Find peaks
            threshold = min_amplitude * np.max(avg_spectrum)
            peaks, properties = find_peaks(
                avg_spectrum,
                height=threshold,
                distance=5  # minimum distance between peaks in frequency bins
            )
            
            if len(peaks) > 0:
                peak_freqs = self.frequencies[peaks]
                peak_amps = avg_spectrum[peaks]
                
                # Find harmonic series
                harmonics = self._find_harmonic_series(
                    peak_freqs, peak_amps, harmonic_tolerance
                )
                
                if harmonics:
                    harmonics_data.append({
                        'time_start': time_chunk[0],
                        'time_end': time_chunk[-1],
                        'harmonics': harmonics
                    })
        
        self.detected_harmonics = harmonics_data
        print(f"Detected {len(harmonics_data)} harmonic segments")
        
        return True
    
    def _find_harmonic_series(self, frequencies, amplitudes, tolerance=0.05):
        """
        Find harmonic series from detected peaks
        
        Parameters:
        frequencies: array of peak frequencies
        amplitudes: array of peak amplitudes
        tolerance: tolerance for harmonic ratios
        """
        harmonics = []
        
        # Sort by amplitude (strongest first)
        sorted_indices = np.argsort(amplitudes)[::-1]
        
        for i, idx in enumerate(sorted_indices):
            f0_candidate = frequencies[idx]
            amp0 = amplitudes[idx]
            
            # Find potential harmonics
            harmonic_freqs = [f0_candidate]
            harmonic_amps = [amp0]
            
            # Check for integer multiples (2f0, 3f0, 4f0, etc.)
            for n in range(2, 8):  # Check up to 7th harmonic
                target_freq = n * f0_candidate
                
                # Find closest frequency
                freq_diffs = np.abs(frequencies - target_freq)
                closest_idx = np.argmin(freq_diffs)
                
                # Check if within tolerance
                if freq_diffs[closest_idx] / target_freq < tolerance:
                    harmonic_freqs.append(frequencies[closest_idx])
                    harmonic_amps.append(amplitudes[closest_idx])
            
            # Only consider as harmonic series if we found at least 2 harmonics
            if len(harmonic_freqs) >= 3:
                harmonics.append({
                    'fundamental': f0_candidate,
                    'frequencies': harmonic_freqs,
                    'amplitudes': harmonic_amps,
                    'n_harmonics': len(harmonic_freqs)
                })
        
        return harmonics

# Add methods to the class
WaveletHarmonicsDetector.detect_harmonics = detect_harmonics
WaveletHarmonicsDetector._find_harmonic_series = _find_harmonic_series

In [ ]:
    def cyclostationary_analysis(self, window_size=4096, overlap=0.75, alpha_max=50):
        """
        Perform cyclo-stationary analysis to detect periodic patterns
        
        Parameters:
        window_size: size of analysis window
        overlap: overlap between windows (0-1)
        alpha_max: maximum cyclic frequency to analyze (Hz)
        """
        if self.audio_data is None:
            print("No audio data loaded.")
            return False
            
        print(f"Computing cyclo-stationary analysis...")
        
        # Compute STFT for cyclic spectral analysis
        hop_length = int(window_size * (1 - overlap))
        
        f_stft, t_stft, stft = signal.stft(
            self.audio_data,
            fs=self.sample_rate,
            window='hann',
            nperseg=window_size,
            noverlap=int(window_size * overlap)
        )
        
        # Compute cyclic spectral density
        magnitude = np.abs(stft)
        
        # Simple cyclic analysis: look for periodicities in the spectrogram
        cyclic_features = []
        
        # For each frequency bin, analyze temporal variations
        for freq_idx in range(len(f_stft)):
            if f_stft[freq_idx] > alpha_max:
                break
                
            time_series = magnitude[freq_idx, :]
            
            # Compute autocorrelation to find periodicities
            autocorr = np.correlate(time_series, time_series, mode='full')
            autocorr = autocorr[len(autocorr)//2:]
            
            # Find peaks in autocorrelation (periodic patterns)
            peaks, _ = find_peaks(autocorr[1:], height=0.3*np.max(autocorr))
            
            if len(peaks) > 0:
                # Convert peak locations to cyclic frequencies
                dt_stft = t_stft[1] - t_stft[0]
                cyclic_freqs = 1.0 / (peaks * dt_stft)
                
                cyclic_features.append({
                    'frequency': f_stft[freq_idx],
                    'cyclic_frequencies': cyclic_freqs,
                    'strengths': autocorr[peaks]
                })
        
        self.cyclic_features = cyclic_features
        print(f"Found {len(cyclic_features)} frequency bins with cyclic patterns")
        
        return True

# Add method to the class
WaveletHarmonicsDetector.cyclostationary_analysis = cyclostationary_analysis

In [ ]:
    def visualize_results(self, time_range=None, freq_range=None, figsize=(20, 15)):
        """
        Create comprehensive visualizations of the analysis results
        
        Parameters:
        time_range: tuple of (start, end) time in seconds
        freq_range: tuple of (min, max) frequency in Hz
        figsize: figure size tuple
        """
        if self.cwt_coeffs is None:
            print("No CWT results to visualize.")
            return
            
        fig, axes = plt.subplots(4, 1, figsize=figsize)
        
        # Apply time and frequency ranges
        if time_range is None:
            time_range = (self.time_vector[0], self.time_vector[-1])
        if freq_range is None:
            freq_range = (self.frequencies.min(), self.frequencies.max())
            
        t_mask = (self.time_vector >= time_range[0]) & (self.time_vector <= time_range[1])
        f_mask = (self.frequencies >= freq_range[0]) & (self.frequencies <= freq_range[1])
        
        time_plot = self.time_vector[t_mask]
        freq_plot = self.frequencies[f_mask]
        cwt_plot = self.cwt_coeffs[np.ix_(f_mask, t_mask)]
        audio_plot = self.audio_data[t_mask]
        
        # 1. Time domain signal
        axes[0].plot(time_plot, audio_plot, 'b-', alpha=0.7)
        axes[0].set_ylabel('Amplitude')
        axes[0].set_title('Underwater Acoustic Signal - Time Domain')
        axes[0].grid(True, alpha=0.3)
        
        # 2. CWT Scalogram
        magnitude_db = 20 * np.log10(np.abs(cwt_plot) + 1e-12)
        im = axes[1].pcolormesh(
            time_plot, freq_plot, magnitude_db, 
            shading='gouraud', cmap='viridis'
        )
        axes[1].set_ylabel('Frequency (Hz)')
        axes[1].set_title('Continuous Wavelet Transform - Scalogram')
        plt.colorbar(im, ax=axes[1], label='Magnitude (dB)')
        
        # Overlay detected harmonics
        for harm_segment in self.detected_harmonics:
            t_start, t_end = harm_segment['time_start'], harm_segment['time_end']
            if t_start >= time_range[0] and t_end <= time_range[1]:
                for harmonic in harm_segment['harmonics']:
                    freqs = harmonic['frequencies']
                    for f in freqs:
                        if freq_range[0] <= f <= freq_range[1]:
                            axes[1].hlines(f, t_start, t_end, colors='red', linewidth=2, alpha=0.8)
        
        # 3. Average spectrum with harmonic markers
        avg_spectrum = np.mean(np.abs(cwt_plot), axis=1)
        axes[2].semilogy(freq_plot, avg_spectrum, 'b-', linewidth=1.5)
        axes[2].set_ylabel('Average Magnitude')
        axes[2].set_title('Average Frequency Spectrum with Detected Harmonics')
        axes[2].grid(True, alpha=0.3)
        
        # Mark detected harmonics
        all_harmonic_freqs = []
        for harm_segment in self.detected_harmonics:
            for harmonic in harm_segment['harmonics']:
                all_harmonic_freqs.extend(harmonic['frequencies'])
        
        if all_harmonic_freqs:
            unique_freqs = np.unique(all_harmonic_freqs)
            for f in unique_freqs:
                if freq_range[0] <= f <= freq_range[1]:
                    axes[2].axvline(f, color='red', alpha=0.7, linestyle='--')
        
        # 4. Harmonic series summary
        axes[3].axis('off')
        
        # Create summary text
        summary_text = "Detected Harmonic Series:\n\n"
        
        for i, harm_segment in enumerate(self.detected_harmonics[:5]):  # Show first 5
            summary_text += f"Segment {i+1}: {harm_segment['time_start']:.1f}-{harm_segment['time_end']:.1f}s\n"
            for j, harmonic in enumerate(harm_segment['harmonics'][:3]):  # Show first 3 per segment
                f0 = harmonic['fundamental']
                n_harm = harmonic['n_harmonics']
                summary_text += f"  Series {j+1}: f₀={f0:.1f}Hz, {n_harm} harmonics\n"
                
                # Show first few harmonics
                freqs_str = ", ".join([f"{f:.1f}" for f in harmonic['frequencies'][:4]])
                summary_text += f"    Frequencies: {freqs_str}...\n"
            summary_text += "\n"
        
        if len(self.detected_harmonics) > 5:
            summary_text += f"... and {len(self.detected_harmonics) - 5} more segments\n"
            
        axes[3].text(0.05, 0.95, summary_text, transform=axes[3].transAxes, 
                    fontsize=10, verticalalignment='top', fontfamily='monospace',
                    bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.5))
        
        axes[3].set_xlabel('Time (s)')
        
        plt.tight_layout()
        plt.show()
        
        # Additional plot: Harmonic evolution over time
        self._plot_harmonic_evolution(time_range, freq_range)
        
    def _plot_harmonic_evolution(self, time_range, freq_range):
        """
        Plot evolution of harmonic frequencies over time
        """
        if not self.detected_harmonics:
            return
            
        plt.figure(figsize=(15, 8))
        
        colors = plt.cm.Set3(np.linspace(0, 1, 12))
        
        for i, harm_segment in enumerate(self.detected_harmonics):
            t_center = (harm_segment['time_start'] + harm_segment['time_end']) / 2
            
            for j, harmonic in enumerate(harm_segment['harmonics']):
                freqs = harmonic['frequencies']
                amps = harmonic['amplitudes']
                
                # Plot each harmonic with size proportional to amplitude
                for k, (f, a) in enumerate(zip(freqs, amps)):
                    if freq_range[0] <= f <= freq_range[1]:
                        size = 50 + 200 * (a / np.max(amps))
                        plt.scatter(t_center, f, s=size, c=[colors[k % len(colors)]], 
                                  alpha=0.7, edgecolors='black', linewidth=0.5)
        
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        plt.title('Harmonic Frequency Evolution Over Time')
        plt.grid(True, alpha=0.3)
        
        # Add legend
        legend_elements = [plt.scatter([], [], s=100, c=[colors[i]], alpha=0.7, 
                                     label=f'Harmonic {i+1}') for i in range(min(6, len(colors)))]
        plt.legend(handles=legend_elements, loc='upper right')
        
        plt.tight_layout()
        plt.show()

# Add methods to the class
WaveletHarmonicsDetector.visualize_results = visualize_results
WaveletHarmonicsDetector._plot_harmonic_evolution = _plot_harmonic_evolution

## Example Analysis: Scooter/DPV Detection

Now let's apply our wavelet harmonics detector to analyze the scooter example data and visualize the results.

In [ ]:
# Initialize detector
detector = WaveletHarmonicsDetector(sample_rate=192000)

# Load scooter example data (limit to first 30 seconds for demo)
audio_file = "/Users/itamarmerfeld/Projects/UATR/data/scooter_example_1.wav"
success = detector.load_and_preprocess(
    audio_file, 
    duration=30,  # Limit to 30 seconds for faster processing
    bandpass_filter=True,
    low_freq=20,   # Focus on scooter frequency range
    high_freq=2000
)

if success:
    print("Audio loaded successfully!")
else:
    print("Failed to load audio file")

In [ ]:
# Compute Continuous Wavelet Transform
if detector.audio_data is not None:
    success = detector.compute_cwt(
        wavelet='cmor1.5-1.0',  # Complex Morlet wavelet
        freq_range=(20, 2000)   # Focus on scooter signature range
    )
    
    if success:
        print("CWT computation completed!")
    else:
        print("CWT computation failed")

In [ ]:
# Detect harmonic series
if detector.cwt_coeffs is not None:
    success = detector.detect_harmonics(
        min_amplitude=0.15,      # 15% of maximum amplitude
        min_duration=1.0,        # Minimum 1 second duration
        harmonic_tolerance=0.08  # 8% tolerance for harmonic ratios
    )
    
    if success:
        print("Harmonic detection completed!")
        print(f"Total harmonic segments found: {len(detector.detected_harmonics)}")
    else:
        print("Harmonic detection failed")

In [ ]:
# Perform cyclo-stationary analysis
if detector.audio_data is not None:
    success = detector.cyclostationary_analysis(
        window_size=8192,   # Larger window for better frequency resolution
        overlap=0.8,        # High overlap for smooth analysis
        alpha_max=100       # Check up to 100 Hz cyclic frequencies
    )
    
    if success:
        print("Cyclo-stationary analysis completed!")
        if hasattr(detector, 'cyclic_features'):
            print(f"Found cyclic patterns in {len(detector.cyclic_features)} frequency bins")
    else:
        print("Cyclo-stationary analysis failed")

In [ ]:
# Create comprehensive visualizations
if detector.cwt_coeffs is not None:
    print("Creating visualizations...")
    detector.visualize_results(
        time_range=(0, 30),      # Full 30-second range
        freq_range=(20, 1000),   # Focus on main scooter frequencies
        figsize=(20, 16)
    )

In [ ]:
# Print detailed analysis summary
print("=== UNDERWATER ACOUSTIC HARMONICS ANALYSIS SUMMARY ===")
print(f"Audio Duration: {len(detector.audio_data)/detector.sample_rate:.1f} seconds")
print(f"Sample Rate: {detector.sample_rate} Hz")
print(f"Frequency Analysis Range: {detector.frequencies.min():.1f} - {detector.frequencies.max():.1f} Hz")
print(f"\nHARMONIC DETECTION RESULTS:")
print(f"Total Segments with Harmonics: {len(detector.detected_harmonics)}")

if detector.detected_harmonics:
    total_harmonics = sum(len(seg['harmonics']) for seg in detector.detected_harmonics)
    print(f"Total Harmonic Series Detected: {total_harmonics}")
    
    # Analyze fundamental frequencies
    all_fundamentals = []
    for segment in detector.detected_harmonics:
        for harmonic in segment['harmonics']:
            all_fundamentals.append(harmonic['fundamental'])
    
    if all_fundamentals:
        print(f"\nFUNDAMENTAL FREQUENCY STATISTICS:")
        print(f"Range: {np.min(all_fundamentals):.1f} - {np.max(all_fundamentals):.1f} Hz")
        print(f"Mean: {np.mean(all_fundamentals):.1f} Hz")
        print(f"Std: {np.std(all_fundamentals):.1f} Hz")
        
        # Common fundamental frequencies (potential scooter signatures)
        unique_f0, counts = np.unique(np.round(all_fundamentals, 0), return_counts=True)
        top_f0_idx = np.argsort(counts)[-5:][::-1]  # Top 5 most common
        
        print(f"\nMOST COMMON FUNDAMENTAL FREQUENCIES (Potential Scooter Signatures):")
        for i in top_f0_idx:
            if i < len(unique_f0):
                print(f"  {unique_f0[i]:.0f} Hz: detected {counts[i]} times")

# Cyclo-stationary results
if hasattr(detector, 'cyclic_features') and detector.cyclic_features:
    print(f"\nCYCLO-STATIONARY ANALYSIS:")
    print(f"Frequency bins with cyclic patterns: {len(detector.cyclic_features)}")
    
    # Summarize cyclic frequencies
    all_cyclic_freqs = []
    for feature in detector.cyclic_features:
        all_cyclic_freqs.extend(feature['cyclic_frequencies'])
    
    if all_cyclic_freqs:
        print(f"Detected cyclic frequencies: {np.min(all_cyclic_freqs):.2f} - {np.max(all_cyclic_freqs):.2f} Hz")
        print(f"Mean cyclic frequency: {np.mean(all_cyclic_freqs):.2f} Hz")

print(f"\n=== ANALYSIS COMPLETE ===")